# 태스크 2: A/B 테스트

이 노트북에서는 A/B 테스트 전략을 구성하고, 모델 변형을 테스트하고, 모델 변형의 성능을 검토합니다.

## 태스크 2.1: 환경 설정

필요한 라이브러리와 종속성을 설치합니다.

In [ ]:
%%capture
%matplotlib inline
from datetime import datetime, timedelta
import time
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.image_uris import retrieve
from sagemaker.session import production_variant
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

region = boto3.Session().region_name
role = get_execution_role()
sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")
cw = boto3.Session().client("cloudwatch")

bucket = sm_session.default_bucket()
prefix = 'sagemaker/abalone'

## 태스크 2.2: 엔드포인트 만들기

회사에서 방금 테스트하려는 새 모델을 만들었습니다. 본 실습에서는 프로덕션 단계의 현재 모델과 함께 새 모델을 Amazon Simple Storage Service(S3) 버킷에 업로드합니다. 그런 다음 모델 변형을 만듭니다. 마지막으로 프로덕션 변형을 사용하여 엔드포인트를 만듭니다.

먼저 모델을 Amazon S3 버킷에 업로드합니다. 현재 프로덕션 단계의 모델은 *model_A.tar.gz*이고 새 모델은 *model_B.tar.gz*입니다.

In [ ]:
#upload-models
model_url = S3Uploader.upload(
    local_path="models/model_A.tar.gz", desired_s3_uri=f"s3://{bucket}/{prefix}"
)
model_url2 = S3Uploader.upload(
    local_path="models/model_B.tar.gz", desired_s3_uri=f"s3://{bucket}/{prefix}"
)
model_url, model_url2

다음으로 사전 훈련된 전복 모델에 대한 모델 정의를 만듭니다.

In [ ]:
#create-model-definitions
model_name = f"abalone-A-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name2 = f"abalone-B-{datetime.now():%Y-%m-%d-%H-%M-%S}"
image_uri = retrieve("xgboost", boto3.Session().region_name, "1.5-1")
image_uri2 = retrieve("xgboost", boto3.Session().region_name, "1.5-1")

response = sm_session.create_model(
    name=model_name, role=role, container_defs={"Image": image_uri, "ModelDataUrl": model_url}
)

response_2 = sm_session.create_model(
    name=model_name2, role=role, container_defs={"Image": image_uri2, "ModelDataUrl": model_url2}
)

print(response)
print(response_2)

그런 다음 각각 고유한 모델이 포함된 두 가지 변형을 만듭니다. 요청의 50%가 **Production-Model-A**로 이동하고 50%가 **New-Model-B**로 이동하도록 두 변형 모두에 대해 *initial_weight*를 1로 설정합니다.

프로덕션 변형을 만들려면 다음 옵션을 구성해야 합니다.
- **model_name**: 호스팅하려는 모델의 이름입니다. 모델을 만들 때 이 이름을 지정했습니다.
- **instance_type**: 컴퓨팅 인스턴스 유형입니다.
- **initial_instance_count**: 처음 시작할 인스턴스의 수입니다.
- **variant_name**: 프로덕션 변형(프로덕션 단계의 하나 이상의 모델)의 이름입니다.
- **initial_weight**: 각 변형의 시작 가중치입니다.

In [ ]:
#create-production-variants
variant1 = production_variant(
    model_name=model_name,
    instance_type="ml.m5.large",
    initial_instance_count=4,
    variant_name="Production-Model-A",
    initial_weight=1,
)
variant2 = production_variant(
    model_name=model_name2,
    instance_type="ml.m5.large",
    initial_instance_count=4,
    variant_name="New-Model-B",
    initial_weight=1,
)

(variant1, variant2)

마지막으로 두 가지 변형을 사용하여 엔드포인트를 만듭니다.

<i class="fas fa-sticky-note" style="color:#ff6633"></i> **참고:** 엔드포인트를 만드는 데 4~5분 정도 걸립니다.

In [ ]:
#create-endpoint
endpoint_name = f"Abalone-{datetime.now():%Y-%m-%d-%H-%M-%S}"

sm_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)

셀이 완성되면 *Abalone-2025-01-01-01-01-00*과 같은 엔드포인트 이름이 반환됩니다.

## 태스크 2.3: 호출 지표 평가

Amazon SageMaker는 Amazon CloudWatch의 각 변형에 대한 지연 시간과 호출과 같은 지표를 내보냅니다. 엔드포인트를 테스트하려면 배포된 모델을 호출하고 호출 지표를 평가합니다. 이전 태스크에서 만든 엔드포인트로 데이터를 보내 실시간으로 추론을 얻을 수 있습니다. 데이터 과학 팀이 새로운 데이터 집합을 받아서 당신에게 보냈습니다. 이 새 데이터를 사용하여 모델을 테스트합니다.

먼저 호출 샘플에 대한 테스트 데이터의 부분 집합을 가져옵니다.

In [ ]:
#import-data
shape=pd.read_csv("data/abalone_data_test.csv", header=0)
shape.sample(5)

그런 다음 **invoke_endpoint**를 사용하여 엔드포인트로 트래픽을 보냅니다.

<i class="fas fa-sticky-note" style="color:#ff6633"></i> **참고:** 엔드포인트 호출은 완료하는 데 1~2분 정도 걸립니다.

In [ ]:
#send-test-data
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")

with open("data/abalone_data_test.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=payload)
f.close()
print("Done!")

호출이 변형 간에 어떻게 분할되는지 보려면 CloudWatch에서 변형별 호출 수를 쿼리합니다.

In [ ]:
#get-cloudwatch-metrics
def get_invocation_metrics_for_endpoint_variant(endpoint_name, variant_name, start_time, end_time):
    metrics = cw.get_metric_statistics(
        Namespace="AWS/SageMaker",
        MetricName="Invocations",
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=["Sum"],
        Dimensions=[
            {"Name": "EndpointName", "Value": endpoint_name},
            {"Name": "VariantName", "Value": variant_name},
        ],
    )
    return (
        pd.DataFrame(metrics["Datapoints"])
        .sort_values("Timestamp")
        .set_index("Timestamp")
        .drop("Unit", axis=1)
        .rename(columns={"Sum": variant_name})
    )


def plot_endpoint_metrics(start_time=None):
    start_time = start_time or datetime.now() - timedelta(minutes=60)
    end_time = datetime.now()
    metrics_variant1 = get_invocation_metrics_for_endpoint_variant(
        endpoint_name, variant1["VariantName"], start_time, end_time
    )
    metrics_variant2 = get_invocation_metrics_for_endpoint_variant(
        endpoint_name, variant2["VariantName"], start_time, end_time
    )
    metrics_variants = metrics_variant1.join(metrics_variant2, how="outer")
    metrics_variants.plot()
    return metrics_variants

print("Waiting a minute for initial metric creation...")
time.sleep(60)
plot_endpoint_metrics()

이 표는 프로덕션 모델인 모델 A의 호출 수를 보고합니다. 또한 새 모델인 모델 B의 호출 수도 보고합니다. 이 차트는 일정 기간 동안의 모델 변형별 호출 수를 표시합니다. 표와 차트는 트래픽이 두 가지 변형 간에 고르게 분산되어 있음을 나타냅니다.

A/B 테스트를 수행하는 경우 *invoke_endpoint* 호출 시 *TargetVariant*를 지정하여 특정 변형을 호출할 수 있습니다. 현재 프로덕션 단계에 있는 변형인 **Production-Model-A**를 호출합니다.

In [ ]:
#get-model-A-predictions
predictions = ""

print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
with open("data/abalone_data_test.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
            TargetVariant=variant1["VariantName"],
        )
        predictions = ",".join([predictions, response["Body"].read().decode("utf-8")])
f.close()

# Convert the predictions to a numpy array and round the values
pred_np = np.fromstring(predictions[1:], sep=",")
preds = np.round(pred_np)

print("Done!")

변형별 트래픽을 다시 확인합니다. 이번에는 모든 호출이 *Production-Model-A*를 대상으로 했기 때문에 해당 변형에 대해서만 호출 수가 증가했음을 알 수 있습니다.

In [ ]:
#plot-model-A-metrics
time.sleep(20)  # let metrics catch up
plot_endpoint_metrics()

## 태스크 2.4: 프로덕션 데이터를 사용한 모델 평가

새 모델이 현재 프로덕션 단계인 모델보다 개선되었는지 확인하려면 두 모델을 모두 평가하고 결과를 비교합니다. 예측 값 목록을 사용하여 각 변형의 R 제곱 점수(R2 점수), 평균 절대 오차(MAE), 평균 제곱근 오차(RMSE) 지표를 평가할 수 있습니다. 이러한 지표는 새 모델이 현재 프로덕션 단계에 있는 모델보다 개선되었는지 여부를 판단하는 데 도움이 됩니다.

지표 정의:
- **R2 점수:** R2 점수는 선택한 모델의 적합도를 수평 직선(귀무가설)의 적합도와 비교한 값입니다. R2 점수의 범위는 -1에서 1까지입니다. R2 점수가 음수이면 모델이 데이터에 적합하지 않음을 의미합니다. 양수가 더 클수록 더 적합함을 의미합니다.
- **MAE:** 실제 값과 예측 값의 차이를 데이터 집합의 총 포인트 수로 나눈 값입니다. 이는 예측이 평균적으로 실제 모델에 얼마나 가까운지 파악하는 데 도움이 됩니다.
- **RMSE:** 실제 값과 예측 값 간의 차이의 제곱근입니다. 예측에 큰 오차(과대평가 또는 과소평가)가 있는지 여부를 판단하는 데 도움이 됩니다.

참고로 모델은 일련의 특성을 기반으로 전복의 고리 수를 예측합니다. 이러한 예측은 전복에 손상을 주지 않으면서 전복의 나이를 추정하는 데 사용됩니다. 각 고리는 나무의 나이테와 마찬가지로 1살에 해당합니다. 예측 값은 모델이 추정하는 전복의 고리 수입니다.

시작하려면 이전 태스크에서 수집된 예측 값을 사용하여 첫 번째 변형인 **Production-Model-A**를 평가합니다. 

In [ ]:
#evaluate-model-A-metrics
df_with_labels = pd.read_csv("data/abalone_data_new.csv", header=None)
test_labels = df_with_labels.iloc[:, 0]
labels = test_labels.to_numpy()
print("The predicted values are:\n {}\n".format(preds))
print("The actual values are:\n {}\n".format(labels))

# Calculate R2
score = r2_score(labels, preds)
print("The R2 score of the current production model is {}".format(round(score, 2)))

# Calculate MAE
score = mean_absolute_error(labels, preds)
print("The Mean Absolute Error of the current production model is {}".format(round(score, 2)))

# Calculate RMSE
score = np.sqrt(mean_absolute_error(labels, preds))
print("The Root Mean Squared Error of the current production model is {}".format(round(score, 2)))

현재 프로덕션 모델을 성공적으로 평가했습니다. 태스크는 이러한 지표를 사용하여 새 모델이 이러한 점수를 향상시키는지 여부를 결정하는 것입니다. 새 모델이 프로덕션 모델보다 우수하면 R2 점수가 증가하고, MAE가 감소하며, RMSE도 감소합니다. 본 태스크의 뒷부분에서 모델 지표 간의 차이를 평가합니다.

새 모델의 예측을 확인하려면 **New-Model-B** 변형을 호출합니다.

In [ ]:
predictions2 = ""
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
with open("data/abalone_data_test.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
            TargetVariant=variant2["VariantName"],
        )
        predictions2 = ",".join([predictions2, response["Body"].read().decode("utf-8")])

# Convert the predictions to a numpy array and round the values
pred_np2 = np.fromstring(predictions2[1:], sep=",")
preds2 = np.round(pred_np2)

print("Done!")

변형별 트래픽을 다시 확인합니다. 이번에는 모든 호출이 *New-Model-B*를 대상으로 했기 때문에 해당 변형에 대해서만 호출 수가 증가했음을 알 수 있습니다.

In [ ]:
#plot-model-B-metrics
time.sleep(20)  # let metrics catch up
plot_endpoint_metrics()

이제 두 번째 모델에 대한 예측 값의 목록을 수집했으므로 R2 점수, MAE, RMSE 지표를 평가합니다. 

In [ ]:
#evaluate-model-B-metrics
df_with_labels = pd.read_csv("data/abalone_data_new.csv", header=None)
test_labels = df_with_labels.iloc[:, 0]
labels = test_labels.to_numpy()
print("The predicted values are:\n {}\n".format(preds2))
print("The actual values are:\n {}\n".format(labels))

# Calculate R2
score = r2_score(labels, preds2)
print("The R2 score of the new model is {}".format(round(score, 2)))

# Calculate MAE
score = mean_absolute_error(labels, preds2)
print("The Mean Absolute Error of the new model is {}".format(round(score, 2)))

# Calculate RMSE
score = np.sqrt(mean_absolute_error(labels, preds2))
print("The Root Mean Squared Error of the new model is {}".format(round(score, 2)))

이 모델은 현재 프로덕션 모델보다 성능이 좋지 않습니다. R2 점수는 더 낮고, MAE는 더 높으며, RMSE도 더 높습니다. 테스트할 새 모델이 있으면 엔드포인트 구성을 수정하여 프로덕션 트래픽을 해당 모델로 이동하기 전에 새 모델을 배포할 수 있습니다.

성공적으로 A/B 테스트 전략을 구성하고, 모델 변형을 테스트하고, 모델 변형의 성능을 검토했습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **Task 3** 부분을 계속 진행합니다.